In [3]:
import pandas as pd
import spacy
nlp = spacy.load('models/drug_names')

In [4]:
#Create a dataframe with the existing data (to mimic actual application)
df = pd.DataFrame(['30mg Oral Tablet', 'Tylenol 10mg Tablet', 'Ibuprofen Tablet'], columns=['drug_name']) #What about drug names without quanities & mechanisms?
df['docs'] = df.apply(lambda x: [nlp(i) for i in x])

#Create a dictionary to save entities in
ner_dict = {'DRUGNAME': [], 'QUANTITY': [], 'MECHANISM': []}


def get_entities(df):
    for name in df['docs']:
        #If the entity was predicted NOT to exist by Ner in the drug name
        annotations_found = [i.label_ for i in name.ents]
        if 'DRUGNAME' not in annotations_found:
            print("'DRUGNAME' doesn't exist for:   {}".format(name.text))
            ner_dict['DRUGNAME'].append(None)
        elif 'QUANTITY' not in annotations_found:
            print("'QUANTITY' doesn't exist for:   {}".format(name.text))
            ner_dict['QUANTITY'].append(None)  
        elif 'MECHANISM' not in annotations_found:
            print("'MECHANISM' doesn't exist for:   {}".format(name.text))
            ner_dict['MECHANISM'].append(None)  

        #If the entity was predicted to exist by Ner in the drug name
        for ent in name.ents:         
            if (ent.label_=='DRUGNAME'):
                ner_dict[ent.label_].append(ent.text)        

            if (ent.label_=='QUANTITY'):
                ner_dict[ent.label_].append(ent.text)  

            if (ent.label_=='MECHANISM'):
                ner_dict[ent.label_].append(ent.text)    

    return ner_dict

ner_dict = get_entities(df)
ner_dict

'DRUGNAME' doesn't exist for:   30mg Oral Tablet
'QUANTITY' doesn't exist for:   Ibuprofen Tablet


{'DRUGNAME': [None, 'Tylenol', 'Ibuprofen'],
 'QUANTITY': ['30mg', '10mg', None],
 'MECHANISM': ['Oral Tablet', 'Tablet', 'Tablet']}

In [5]:
# ner_dict['drugname'] = ner_dict['DRUGNAME']
# del ner_dict['DRUGNAME']
df.join(pd.DataFrame(ner_dict))

,drug_name,docs,DRUGNAME,QUANTITY,MECHANISM
0,30mg Oral Tablet,"(30, mg, Oral, Tablet)",None,30mg,Oral Tablet
1,Tylenol 10mg Tablet,"(Tylenol, 10, mg, Tablet)",Tylenol,10mg,Tablet
2,Ibuprofen Tablet,"(Ibuprofen, Tablet)",Ibuprofen,None,Tablet
